In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Dropout, LSTM
from tensorflow.python.keras import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import OrdinalEncoder
from pandas import DataFrame
from pandas import concat
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score

In [2]:
house_data = pd.read_csv("../preprocessed_data/oneHot_data.csv")

In [3]:
house_data.head()

,hour_sin,hour_cos,month_sin,month_cos,day_sin,day_cos,Month,DayOfWeek,Day,HourofDay,...,T_stp_heat,Humidity,auxHeat1,fan,Thermostat_Temperature,Thermostat_Motion,Remote_Sensor_1_Temperature,Remote_Sensor_1_Motion,T_out,RH_out
0,0.0,1.0,0.0,1.0,0.755303,0.484974,1,1,1,0,...,0.663321,0.177458,0,11,0.330465,0.0,67.0,0.0,0.009112,1.808865
1,0.0,1.0,0.0,1.0,0.755303,0.484974,1,1,1,0,...,0.663321,0.177458,2,9,0.330465,0.0,67.0,0.0,0.009112,1.808865
2,0.0,1.0,0.0,1.0,0.755303,0.484974,1,1,1,0,...,0.663321,0.177458,20,20,0.330465,0.0,67.0,0.0,0.009112,1.808865
3,0.0,1.0,0.0,1.0,0.755303,0.484974,1,1,1,0,...,0.663321,0.177458,9,12,0.330465,0.0,67.0,0.0,0.009112,1.808865
4,0.0,1.0,0.0,1.0,0.755303,0.484974,1,1,1,0,...,0.663321,0.177458,0,8,0.330465,0.0,67.0,0.0,0.009112,1.808865


In [4]:
house_data.columns

Index(['hour_sin', 'hour_cos', 'month_sin', 'month_cos', 'day_sin', 'day_cos',
       'Month', 'DayOfWeek', 'Day', 'HourofDay', 'HvacMode_heat', 'Event_Hold',
       'Event_None', 'Schedule_Away', 'Schedule_Home', 'Schedule_Sleep',
       'T_ctrl', 'T_stp_heat', 'Humidity', 'auxHeat1', 'fan',
       'Thermostat_Temperature', 'Thermostat_Motion',
       'Remote_Sensor_1_Temperature', 'Remote_Sensor_1_Motion', 'T_out',
       'RH_out'],
      dtype='object')

In [5]:
working_df = house_data[['T_ctrl', 'T_stp_heat', 'auxHeat1', 'T_out', 'RH_out']]
cols = ['T_ctrl', 'T_stp_heat', 'T_out', 'RH_out', 'auxHeat1']
working_df = working_df[cols]

In [6]:
working_df.head()

,T_ctrl,T_stp_heat,T_out,RH_out,auxHeat1
0,0.311691,0.663321,0.009112,1.808865,0
1,0.311691,0.663321,0.009112,1.808865,2
2,0.311691,0.663321,0.009112,1.808865,20
3,0.311691,0.663321,0.009112,1.808865,9
4,0.311691,0.663321,0.009112,1.808865,0


In [7]:
def series_to_supervised(data, n_in = 1, n_out = 1, dropnan = True):
    
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

### Model House

In [8]:
reframed = series_to_supervised(working_df, 6, 1)

In [9]:
reframed

,var1(t-6),var2(t-6),var3(t-6),var4(t-6),var5(t-6),var1(t-5),var2(t-5),var3(t-5),var4(t-5),var5(t-5),...,var1(t-1),var2(t-1),var3(t-1),var4(t-1),var5(t-1),var1(t),var2(t),var3(t),var4(t),var5(t)
6,0.311691,0.663321,0.009112,1.808865,0.0,0.311691,0.663321,0.009112,1.808865,2.0,...,0.311691,0.663321,0.009112,1.808865,0.0,0.311691,0.663321,0.075007,1.877919,0
7,0.311691,0.663321,0.009112,1.808865,2.0,0.311691,0.663321,0.009112,1.808865,20.0,...,0.311691,0.663321,0.075007,1.877919,0.0,0.311691,0.663321,0.075007,1.877919,0
8,0.311691,0.663321,0.009112,1.808865,20.0,0.311691,0.663321,0.009112,1.808865,9.0,...,0.311691,0.663321,0.075007,1.877919,0.0,0.311691,0.663321,0.075007,1.877919,14
9,0.311691,0.663321,0.009112,1.808865,9.0,0.311691,0.663321,0.009112,1.808865,0.0,...,0.311691,0.663321,0.075007,1.877919,14.0,0.311691,0.663321,0.075007,1.877919,19
10,0.311691,0.663321,0.009112,1.808865,0.0,0.311691,0.663321,0.009112,1.808865,0.0,...,0.311691,0.663321,0.075007,1.877919,19.0,0.311691,0.663321,0.075007,1.877919,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58832,0.311691,0.663321,-0.649837,-0.055597,10.0,0.311691,0.663321,-0.649837,-0.055597,20.0,...,0.311691,0.663321,-0.649837,-0.124651,0.0,0.311691,0.663321,-0.649837,-0.124651,0
58833,0.311691,0.663321,-0.649837,-0.055597,20.0,0.311691,0.663321,-0.649837,-0.055597,20.0,...,0.311691,0.663321,-0.649837,-0.124651,0.0,0.311691,0.663321,-0.649837,-0.124651,0
58834,0.311691,0.663321,-0.649837,-0.055597,20.0,0.311691,0.663321,-0.649837,-0.055597,9.0,...,0.311691,0.663321,-0.649837,-0.124651,0.0,0.311691,0.663321,-0.649837,-0.124651,0
58835,0.311691,0.663321,-0.649837,-0.055597,9.0,0.311691,0.663321,-0.649837,-0.055597,0.0,...,0.311691,0.663321,-0.649837,-0.124651,0.0,0.311691,0.663321,-0.649837,-0.124651,0


In [10]:
y_auxHeat = reframed['var5(t)']
y_auxHeat = to_categorical(y_auxHeat)
X = reframed.drop(labels = ["var5(t)"], axis = 1)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y_auxHeat, test_size = 0.2, shuffle = True)

In [12]:
X_train = X_train.values
X_test = X_test.values

In [13]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(47064, 34) (47064, 21) (11767, 34) (11767, 21)


In [14]:
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu', input_dim = 34))

# Adding the second hidden layer
classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 21, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 

# Fitting the ANN to the Training set
history = classifier.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 64, epochs = 50, shuffle = True)

Epoch 1/50
736/736 [==============================] - 3s 4ms/step - loss: 0.9796 - accuracy: 0.8167 - val_loss: 0.7255 - val_accuracy: 0.8518
Epoch 2/50
736/736 [==============================] - 3s 3ms/step - loss: 0.7116 - accuracy: 0.8566 - val_loss: 0.6859 - val_accuracy: 0.8622
Epoch 3/50
736/736 [==============================] - 3s 3ms/step - loss: 0.6803 - accuracy: 0.8610 - val_loss: 0.6686 - val_accuracy: 0.8616
Epoch 4/50
736/736 [==============================] - 3s 3ms/step - loss: 0.6614 - accuracy: 0.8629 - val_loss: 0.6538 - val_accuracy: 0.8633
Epoch 5/50
736/736 [==============================] - 3s 3ms/step - loss: 0.6513 - accuracy: 0.8635 - val_loss: 0.6465 - val_accuracy: 0.8652
Epoch 6/50
736/736 [==============================] - 3s 3ms/step - loss: 0.6453 - accuracy: 0.8636 - val_loss: 0.6548 - val_accuracy: 0.8629
Epoch 7/50
736/736 [==============================] - 3s 3ms/step - loss: 0.6428 - accuracy: 0.8632 - val_loss: 0.6372 - val_accuracy: 0.8631
Epoch 

In [ ]:
plt.figure(figsize = (12, 10))
plt.plot(history.history['loss'], label = 'Training Set')
plt.plot(history.history['val_loss'], label = 'Test Set')
plt.legend()
plt.xlabel("Training Epoch Number")
plt.ylabel("Categorical Cross-Entropy Loss")
plt.xlim((-1, 20))
plt.show()

In [ ]:
plt.figure(figsize = (12, 10))
plt.plot(history.history['accuracy'], label = 'Training Set')
plt.plot(history.history['val_accuracy'], label = 'Test Set')
plt.legend()
plt.xlabel("Training Epoch Number")
plt.ylabel("Accuracy Achieved")
plt.xlim((-1, 20))
plt.show()

In [ ]:
y_predicted = classifier.predict(X_test)

In [ ]:
y_predicted = np.argmax(y_predicted, axis = 1)

In [ ]:
y_test = np.argmax(y_test, axis = 1)

In [ ]:
cm = confusion_matrix(y_test, y_predicted)

In [ ]:
cfm_n = np.flipud(cm)
cfm_n = cfm_n.astype('float') / cfm_n.sum(axis=1)[:, np.newaxis]

z = np.round(cfm_n, 2)
#x = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
y = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
#y = ['20', '19', '18', '17', '16', '15', '14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
x = ['20', '19', '18', '17', '16', '15', '14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
#x = ['1', '0']
#y = ['0', '1']
fig = ff.create_annotated_heatmap(z, x = x, y = y, annotation_text = z, colorscale = 'deep')
fig.update_layout(width = 1600, height = 1600, xaxis_showgrid = False, yaxis_showgrid = False, template = 'none')
fig.update_layout(title = {'text': "",
                           'y':0.95,
                           'x':0.47,
                           'xanchor': 'center',
                           'yanchor': 'top'}, 
                  xaxis_title = 'Predicted Labels',
                  yaxis_title = 'True Labels')
fig['layout']['xaxis']['side'] = 'bottom'
fig.show()

In [ ]:
test_sum = 0
for i in range(len(y_test)):
    test_sum += (y_test[i] * 15)

In [ ]:
test_sum

In [ ]:
predicted_sum = 0
for i in range(len(y_predicted)):
    predicted_sum += (y_predicted[i] * 15)

In [ ]:
predicted_sum

In [ ]:
print("Error Percentage in predicted values is : {} %".format(np.absolute((predicted_sum - test_sum)/test_sum)*100))

In [ ]:
f1_score(y_test, y_predicted, average = 'weighted')

In [ ]:
X[0:5]

In [ ]:
X_small, y_small = X[12480:12492], y_auxHeat[12480:12492]

In [ ]:
print(X_small.shape, y_small.shape)

In [ ]:
X_small = X_small.values

In [ ]:
y_small_predicted = classifier.predict(X_small)

In [ ]:
y_small_predicted = np.argmax(y_small_predicted, axis = 1)

In [ ]:
y_small_test = np.argmax(y_small, axis = 1)

In [ ]:
y_small_predicted

In [ ]:
y_small_test

In [ ]:
test_sum = 0
for i in range(len(y_small_test)):
    test_sum += (y_small_test[i] * 15)

In [ ]:
test_sum

In [ ]:
predicted_sum = 0
for i in range(len(y_small_predicted)):
    predicted_sum += (y_small_predicted[i] * 15)

In [ ]:
predicted_sum

In [ ]:
print("Error Percentage in predicted values is : {} %".format(np.absolute((predicted_sum - test_sum)/test_sum)*100))

# Testing ebfd64742c95ab9f71f8ac8a810624464782d17d

In [ ]:
house_data = pd.read_csv("../preprocessed_data/std_test_data_ebfd64742c95ab9f71f8ac8a810624464782d17d.csv")

In [ ]:
working_df = house_data[['T_ctrl', 'T_stp_heat', 'auxHeat1', 'T_out', 'RH_out']]
cols = ['T_ctrl', 'T_stp_heat', 'T_out', 'RH_out', 'auxHeat1']
working_df = working_df[cols]

In [ ]:
working_df.head()

In [ ]:
reframed = series_to_supervised(working_df, 6, 1)

In [ ]:
y_auxHeat = reframed['var5(t)']
y_auxHeat = to_categorical(y_auxHeat)
X = reframed.drop(labels = ["var5(t)"], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_auxHeat, test_size = 0.2, shuffle = True)

In [ ]:
X_train = X_train.values
X_test = X_test.values

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu', input_dim = 34))

# Adding the second hidden layer
classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 21, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 

# Fitting the ANN to the Training set
history = classifier.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 10, epochs = 20, shuffle = True)

In [ ]:
plt.plot(history.history['loss'], label = 'train')
plt.plot(history.history['val_loss'], label = 'test')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label = 'train')
plt.plot(history.history['val_accuracy'], label = 'test')
plt.legend()
plt.show()

In [ ]:
y_predicted = classifier.predict(X_test)

In [ ]:
y_predicted = np.argmax(y_predicted, axis = 1)

In [ ]:
y_test = np.argmax(y_test, axis = 1)

In [ ]:
cm = confusion_matrix(y_test, y_predicted)

cfm_n = np.flipud(cm)
cfm_n = cfm_n.astype('float') / cfm_n.sum(axis=1)[:, np.newaxis]

z = np.round(cfm_n, 2)
#x = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
y = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
#y = ['20', '19', '18', '17', '16', '15', '14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
x = ['20', '19', '18', '17', '16', '15', '14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
#x = ['1', '0']
#y = ['0', '1']
fig = ff.create_annotated_heatmap(z, x = x, y = y, annotation_text = z, colorscale = 'deep')
fig.update_layout(width = 1600, height = 1600, xaxis_showgrid = False, yaxis_showgrid = False, template = 'none')
fig.update_layout(title = {'text': "",
                           'y':0.95,
                           'x':0.47,
                           'xanchor': 'center',
                           'yanchor': 'top'}, 
                  xaxis_title = 'Predicted Labels',
                  yaxis_title = 'True Labels')
fig['layout']['xaxis']['side'] = 'bottom'
fig.show()

In [ ]:
test_sum = 0
for i in range(len(y_test)):
    test_sum += (y_test[i] * 15)

In [ ]:
test_sum

In [ ]:
predicted_sum = 0
for i in range(len(y_predicted)):
    predicted_sum += (y_predicted[i] * 15)

In [ ]:
predicted_sum

In [ ]:
print("Error Percentage in predicted values is : {} %".format(np.absolute((predicted_sum - test_sum)/test_sum)*100))

In [ ]:
f1_score(y_test, y_predicted, average = 'weighted')

In [ ]:
X_small, y_small = X[12465:12477], y_auxHeat[12465:12477]

In [ ]:
print(X_small.shape, y_small.shape)

In [ ]:
X_small = X_small.values

In [ ]:
y_small_predicted = classifier.predict(X_small)

In [ ]:
y_small_predicted = np.argmax(y_small_predicted, axis = 1)

In [ ]:
y_small_test = np.argmax(y_small, axis = 1)

In [ ]:
y_small_predicted

In [ ]:
y_small_test

In [ ]:
test_sum = 0
for i in range(len(y_small_test)):
    test_sum += (y_small_test[i] * 15)

In [ ]:
test_sum

In [ ]:
predicted_sum = 0
for i in range(len(y_small_predicted)):
    predicted_sum += (y_small_predicted[i] * 15)

In [ ]:
predicted_sum

In [ ]:
print("Error Percentage in predicted values is : {} %".format(np.absolute((predicted_sum - test_sum)/test_sum)*100))

# Testing 6150c70f76a3f4b0c6e303de7da9422aceb37345

In [ ]:
house_data = pd.read_csv("../preprocessed_data/std_test_data_6150c70f76a3f4b0c6e303de7da9422aceb37345.csv")

In [ ]:
working_df = house_data[['T_ctrl', 'T_stp_heat', 'auxHeat1', 'T_out', 'RH_out']]
cols = ['T_ctrl', 'T_stp_heat', 'T_out', 'RH_out', 'auxHeat1']
working_df = working_df[cols]

In [ ]:
working_df.head()

In [ ]:
reframed = series_to_supervised(working_df, 6, 1)

In [ ]:
y_auxHeat = reframed['var5(t)']
y_auxHeat = to_categorical(y_auxHeat)
X = reframed.drop(labels = ["var5(t)"], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_auxHeat, test_size = 0.2, shuffle = True)

In [ ]:
X_train = X_train.values
X_test = X_test.values

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu', input_dim = 34))

# Adding the second hidden layer
classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 21, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 

# Fitting the ANN to the Training set
history = classifier.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 10, epochs = 20, shuffle = True)

In [ ]:
plt.plot(history.history['loss'], label = 'train')
plt.plot(history.history['val_loss'], label = 'test')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label = 'train')
plt.plot(history.history['val_accuracy'], label = 'test')
plt.legend()
plt.show()

In [ ]:
y_predicted = classifier.predict(X_test)

In [ ]:
y_predicted = np.argmax(y_predicted, axis = 1)

In [ ]:
y_test = np.argmax(y_test, axis = 1)

In [ ]:
cm = confusion_matrix(y_test, y_predicted)

cfm_n = np.flipud(cm)
cfm_n = cfm_n.astype('float') / cfm_n.sum(axis=1)[:, np.newaxis]

z = np.round(cfm_n, 2)
#x = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
y = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
#y = ['20', '19', '18', '17', '16', '15', '14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
x = ['20', '19', '18', '17', '16', '15', '14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
#x = ['1', '0']
#y = ['0', '1']
fig = ff.create_annotated_heatmap(z, x = x, y = y, annotation_text = z, colorscale = 'deep')
fig.update_layout(width = 1600, height = 1600, xaxis_showgrid = False, yaxis_showgrid = False, template = 'none')
fig.update_layout(title = {'text': "",
                           'y':0.95,
                           'x':0.47,
                           'xanchor': 'center',
                           'yanchor': 'top'}, 
                  xaxis_title = 'Predicted Labels',
                  yaxis_title = 'True Labels')
fig['layout']['xaxis']['side'] = 'bottom'
fig.show()

In [ ]:
test_sum = 0
for i in range(len(y_test)):
    test_sum += (y_test[i] * 15)

In [ ]:
test_sum

In [ ]:
predicted_sum = 0
for i in range(len(y_predicted)):
    predicted_sum += (y_predicted[i] * 15)

In [ ]:
predicted_sum

In [ ]:
print("Error Percentage in predicted values is : {} %".format(np.absolute((predicted_sum - test_sum)/test_sum)*100))

In [ ]:
f1_score(y_test, y_predicted, average = 'weighted')

In [ ]:
X_small, y_small = X[12465:12477], y_auxHeat[12465:12477]

In [ ]:
print(X_small.shape, y_small.shape)

In [ ]:
X_small = X_small.values

In [ ]:
y_small_predicted = classifier.predict(X_small)

In [ ]:
y_small_predicted = np.argmax(y_small_predicted, axis = 1)

In [ ]:
y_small_test = np.argmax(y_small, axis = 1)

In [ ]:
y_small_predicted

In [ ]:
y_small_test

In [ ]:
test_sum = 0
for i in range(len(y_small_test)):
    test_sum += (y_small_test[i] * 15)

In [ ]:
test_sum

In [ ]:
predicted_sum = 0
for i in range(len(y_small_predicted)):
    predicted_sum += (y_small_predicted[i] * 15)

In [ ]:
predicted_sum

In [ ]:
print("Error Percentage in predicted values is : {} %".format(np.absolute((predicted_sum - test_sum)/test_sum)*100))

# Testing 1faa36208e696350ee3ca80197c773f062678051

In [ ]:
house_data = pd.read_csv("../preprocessed_data/std_test_data_1faa36208e696350ee3ca80197c773f062678051.csv")

In [ ]:
working_df = house_data[['T_ctrl', 'T_stp_heat', 'auxHeat1', 'T_out', 'RH_out']]
cols = ['T_ctrl', 'T_stp_heat', 'T_out', 'RH_out', 'auxHeat1']
working_df = working_df[cols]

In [ ]:
working_df.head()

In [ ]:
reframed = series_to_supervised(working_df, 6, 1)

In [ ]:
y_auxHeat = reframed['var5(t)']
y_auxHeat = to_categorical(y_auxHeat)
X = reframed.drop(labels = ["var5(t)"], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_auxHeat, test_size = 0.2, shuffle = True)

In [ ]:
X_train = X_train.values
X_test = X_test.values

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu', input_dim = 34))

# Adding the second hidden layer
classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 21, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 

# Fitting the ANN to the Training set
history = classifier.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 10, epochs = 20, shuffle = True)

In [ ]:
plt.plot(history.history['loss'], label = 'train')
plt.plot(history.history['val_loss'], label = 'test')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label = 'train')
plt.plot(history.history['val_accuracy'], label = 'test')
plt.legend()
plt.show()

In [ ]:
y_predicted = classifier.predict(X_test)

In [ ]:
y_predicted = np.argmax(y_predicted, axis = 1)

In [ ]:
y_test = np.argmax(y_test, axis = 1)

In [ ]:
cm = confusion_matrix(y_test, y_predicted)

cfm_n = np.flipud(cm)
cfm_n = cfm_n.astype('float') / cfm_n.sum(axis=1)[:, np.newaxis]

z = np.round(cfm_n, 2)
#x = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
y = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
#y = ['20', '19', '18', '17', '16', '15', '14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
x = ['20', '19', '18', '17', '16', '15', '14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
#x = ['1', '0']
#y = ['0', '1']
fig = ff.create_annotated_heatmap(z, x = x, y = y, annotation_text = z, colorscale = 'deep')
fig.update_layout(width = 1600, height = 1600, xaxis_showgrid = False, yaxis_showgrid = False, template = 'none')
fig.update_layout(title = {'text': "",
                           'y':0.95,
                           'x':0.47,
                           'xanchor': 'center',
                           'yanchor': 'top'}, 
                  xaxis_title = 'Predicted Labels',
                  yaxis_title = 'True Labels')
fig['layout']['xaxis']['side'] = 'bottom'
fig.show()

In [ ]:
test_sum = 0
for i in range(len(y_test)):
    test_sum += (y_test[i] * 15)

In [ ]:
test_sum

In [ ]:
predicted_sum = 0
for i in range(len(y_predicted)):
    predicted_sum += (y_predicted[i] * 15)

In [ ]:
predicted_sum

In [ ]:
print("Error Percentage in predicted values is : {} %".format(np.absolute((predicted_sum - test_sum)/test_sum)*100))

In [ ]:
f1_score(y_test, y_predicted, average = 'weighted')

In [ ]:
X_small, y_small = X[1234:1246], y_auxHeat[1234:1246]

In [ ]:
print(X_small.shape, y_small.shape)

In [ ]:
y_small_predicted = classifier.predict(X_small)
y_small_predicted = np.argmax(y_small_predicted, axis = 1)

In [ ]:
y_small_predicted

In [ ]:
y_small_test = np.argmax(y_small, axis = 1)

In [ ]:
y_small_test

In [ ]:
test_sum = 0
for i in range(len(y_small_test)):
    test_sum += (y_small_test[i] * 15)

In [ ]:
test_sum

In [ ]:
predicted_sum = 0
for i in range(len(y_small_predicted)):
    predicted_sum += (y_small_predicted[i] * 15)

In [ ]:
predicted_sum

In [ ]:
print("Error Percentage in predicted values is : {} %".format(np.absolute((predicted_sum - test_sum)/test_sum)*100))

# Testing 6c83e118e0c6dde25b3aba4935f902a0291b95a7 Alberta House

In [ ]:
house_data = pd.read_csv("../preprocessed_data/std_test_data_6c83e118e0c6dde25b3aba4935f902a0291b95a7.csv")

In [ ]:
working_df = house_data[['T_ctrl', 'T_stp_heat', 'auxHeat1', 'T_out', 'RH_out']]
cols = ['T_ctrl', 'T_stp_heat', 'T_out', 'RH_out', 'auxHeat1']
working_df = working_df[cols]

In [ ]:
working_df.head()

In [ ]:
reframed = series_to_supervised(working_df, 6, 1)

In [ ]:
y_auxHeat = reframed['var5(t)']
y_auxHeat = to_categorical(y_auxHeat)
X = reframed.drop(labels = ["var5(t)"], axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_auxHeat, test_size = 0.2, shuffle = True)

In [ ]:
X_train = X_train.values

In [ ]:
X_test = X_test.values

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu', input_dim = 34))

# Adding the second hidden layer
classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 21, kernel_initializer = 'uniform', activation = 'softmax'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy']) 

# Fitting the ANN to the Training set
history = classifier.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 10, epochs = 20, shuffle = True)

In [ ]:
plt.plot(history.history['loss'], label = 'train')
plt.plot(history.history['val_loss'], label = 'test')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'], label = 'train')
plt.plot(history.history['val_accuracy'], label = 'test')
plt.legend()
plt.show()

In [ ]:
y_predicted = classifier.predict(X_test)

In [ ]:
y_predicted = np.argmax(y_predicted, axis = 1)

In [ ]:
y_test = np.argmax(y_test, axis = 1)

In [ ]:
cm = confusion_matrix(y_test, y_predicted)

cfm_n = np.flipud(cm)
cfm_n = cfm_n.astype('float') / cfm_n.sum(axis=1)[:, np.newaxis]

z = np.round(cfm_n, 2)
#x = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
y = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
#y = ['20', '19', '18', '17', '16', '15', '14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
x = ['20', '19', '18', '17', '16', '15', '14', '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']
#x = ['1', '0']
#y = ['0', '1']
fig = ff.create_annotated_heatmap(z, x = x, y = y, annotation_text = z, colorscale = 'deep')
fig.update_layout(width = 1600, height = 1600, xaxis_showgrid = False, yaxis_showgrid = False, template = 'none')
fig.update_layout(title = {'text': "",
                           'y':0.95,
                           'x':0.47,
                           'xanchor': 'center',
                           'yanchor': 'top'}, 
                  xaxis_title = 'Predicted Labels',
                  yaxis_title = 'True Labels')
fig['layout']['xaxis']['side'] = 'bottom'
fig.show()

In [ ]:
test_sum = 0
for i in range(len(y_test)):
    test_sum += (y_test[i] * 15)

In [ ]:
test_sum

In [ ]:
predicted_sum = 0
for i in range(len(y_predicted)):
    predicted_sum += (y_predicted[i] * 15)

In [ ]:
predicted_sum

In [ ]:
print("Error Percentage in predicted values is : {} %".format(np.absolute((predicted_sum - test_sum)/test_sum)*100))

In [ ]:
f1_score(y_test, y_predicted, average = 'weighted')

In [ ]:
X_small, y_small = X[12345:12357], y_auxHeat[12345:12357]

In [ ]:
print(X_small.shape, y_small.shape)

In [ ]:
y_small_predicted = classifier.predict(X_small)
y_small_predicted = np.argmax(y_small_predicted, axis = 1)

In [ ]:
y_small_predicted

In [ ]:
y_small_test = np.argmax(y_small, axis = 1)

In [ ]:
y_small_test

In [ ]:
test_sum = 0
for i in range(len(y_small_test)):
    test_sum += (y_small_test[i] * 15)

In [ ]:
test_sum

In [ ]:
predicted_sum = 0
for i in range(len(y_small_predicted)):
    predicted_sum += (y_small_predicted[i] * 15)

In [ ]:
predicted_sum

In [ ]:
print("Error Percentage in predicted values is : {} %".format(np.absolute((predicted_sum - test_sum)/test_sum)*100))